# **Deployment**

In [ ]:
!pip install transformers flask flask-ngrok


In [ ]:
!pip install pyngrok


In [13]:
from pyngrok import ngrok
# Set up ngrok authtoken
ngrok.set_auth_token('ngrok config add-authtoken 2jEyTCV8uUh15aikgko1UXlrhQO_54yHKTPrvcoCZcWcSnoR8')

In [27]:
# Create app.py file
with open("app.py", "w") as f:
    f.write("""
from flask import Flask, request, jsonify, render_template_string
from transformers import pipeline
from pyngrok import ngrok

app = Flask(__name__)

# Set up the ngrok tunnel
ngrok.set_auth_token("2jEyTCV8uUh15aikgko1UXlrhQO_54yHKTPrvcoCZcWcSnoR8")
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:5000'")

model_name = "ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel"
sentiment_pipeline = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)

# Define descriptions for sentiment labels
LABEL_DESCRIPTIONS = {
    "1 star": "🌕🌑🌑🌑🌑 1 Star: Very dissatisfied with the experience.",
    "2 star": "🌕🌕🌑🌑🌑 2 Stars: Dissatisfied with the experience.",
    "3 stars": "🌕🌕🌕🌑🌑 3 Stars: Neutral, neither good nor bad.",
    "4 stars": "🌕🌕🌕🌕🌑 4 Stars: Satisfied with the experience.",
    "5 stars": "🌕🌕🌕🌕🌕 5 Stars: Extremely satisfied with the experience."
}

# HTML template for the form
HTML_FORM = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Sentiment Analysis</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      text-align: center;
      margin: 0;
      padding: 0;
      display: flex;
      justify-content: center;
      align-items: center;
      height: 100vh;
      background-color: #f0f0f0;
    }
    .container {
      background: white;
      padding: 20px;
      border-radius: 10px;
      box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    }
    h1 {
      margin-bottom: 10px;
    }
    p {
      margin: 0;
    }
    form {
      margin-top: 20px;
    }
    textarea {
      width: 100%;
      max-width: 500px;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1>Sentiment Analysis</h1>
    <p>🌘🌗🌖🌕🌔🌓🌒🌑</p>
    <form action="/predict" method="post">
      <label for="review">Write your review here</label><br>
      <p>✍🏻✍🏼✍🏽✍🏾✍🏿⤵️</p>
      <textarea id="review" name="review" rows="4" cols="50">{{ review }}</textarea><br><br>
      <input type="submit" value="Submit">
    </form>
    {% if result %}
      <h2>Prediction Result ➡️ </h2>
      <p>{{ result['label'] }}: {{ result['description'] }}</p>
      <h3>Your Review:</h3>
      <p>{{ review }}</p>
    {% endif %}
  </div>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(HTML_FORM, result=None, review="")

@app.route('/predict', methods=['POST'])
def predict():
    review = request.form.get('review')

    if not review or not isinstance(review, str):
        return render_template_string(HTML_FORM, result={"label": "Error", "description": "Invalid input. Please enter a review."}, review=review)

    # Predict sentiment
    result = sentiment_pipeline([review])[0]
    label = result['label']
    description = LABEL_DESCRIPTIONS.get(label, "Unknown sentiment")

    # Return result to the user
    return render_template_string(HTML_FORM, result={"label": label, "description": description}, review=review)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)  # Make sure Flask runs on port 5000
""")


In [15]:
from transformers import AutoConfig

model_name = "ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel"
config = AutoConfig.from_pretrained(model_name)

print("Labels:", config.label2id)


Labels: {'1 star': 0, '2 star': 1, '3 stars': 2, '4 stars': 3, '5 stars': 4}


In [ ]:
!python app.py